In [3]:
##### Multidimensional EDS biasing using HTF #####
## Similar steps as 1D EDS method. Instead of one CV now we are biasing 2 CVs ## 
## CVs are: 1. average distance between particles and the center of mass along x axis
#           2. average distance between particles and the center of mass along y axis

In [11]:
import hoomd.htf as htf
import tensorflow as tf
import hoomd
import hoomd.md

In [12]:
#### Build training graph for multidimensional EDS ####

def make_eds2D_graph(N, NN, set_pt1,set_pt2):
    #N= Number of atoms in the system, NN=Number of nearest neighbors,
    #set_pt=set point in EDS method
    
    graph =htf.graph_builder(NN,output_forces=True)
    #calculate center of mass
    com = tf.reduce_mean(graph.positions[:, :2], 0)
    #calculate distance of each atom from center of mass along x axis
    r_x = tf.math.subtract(graph.positions[:, 0], tf.ones_like(graph.positions[:, 0])*com[0])
    #calculate distance of each atom from center of mass along y axis
    r_y = tf.math.subtract(graph.positions[:, 1], tf.ones_like(graph.positions[:, 1])*com[1])
    #calculate CV 1: average distance along x axis 
    real_cv1 = tf.reduce_mean(r_x)
    #calculate CV 2: average distance along y axis 
    real_cv2 = tf.reduce_mean(r_y)
    #calculates the running mean of the CVs
    cv_run1=graph.running_mean(tensor=real_cv1,name='cv_run1')
    cv_run2=graph.running_mean(tensor=real_cv2,name='cv_run2')

    #Calculate the EDS alpha value for each CV, every 300 steps. 
    #See htf/utils.py for more information
    eds_alpha1 = htf.eds_bias(real_cv1, set_point=set_pt1, period=300,learning_rate=5.0,name='eds1')
    eds_alpha2 = htf.eds_bias(real_cv2, set_point=set_pt2, period=300,learning_rate=5.0,name='eds2')
    #computes EDS energy
    eds_energy2 = (eds_alpha1 * real_cv1) + (eds_alpha2 * real_cv2)                      
    #compute EDS forces
    eds_forces2 = graph.compute_forces(eds_energy2) 
    
    printer1=tf.Print(cv_run1,[cv_run1],message='CV1')
    printer2=tf.Print(cv_run2,[cv_run2],message='CV2')                     
    printer3=tf.Print(eds_alpha1,[eds_alpha1],message='a1')
    printer4=tf.Print(eds_alpha2,[eds_alpha2],message='a2')
    
    graph.save('my_model2',force_tensor=eds_forces2,out_nodes=[[printer1,500],[printer2,500],[printer4,500],[printer4,500]],virial=None)                     
    

In [13]:
#### Hoomd-Sim code ####

make_eds2D_graph(64, 100, 4.0, 4.5)

with htf.tfcompute('my_model2') as tfcompute:
#with hoomd.htf.tfcompute(model_dir,write_tensorboard=False) as tfcompute: 
    hoomd.context.initialize("--mode=gpu")
    #cut off radius: must be less than the box size
    rcut = 6.0 
    #initialize the lattice
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=2.0),n=[8, 8]) 
    hoomd.md.update.enforce2d()
    nlist = hoomd.md.nlist.cell(check_period=1)
    #enable lj pair potential
    lj = hoomd.md.pair.lj(rcut, nlist) 
    #set lj coefficients
    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0) 
    hoomd.md.integrate.mode_standard(dt=0.005)
    
    # set up NVT simulation
    hoomd.md.integrate.nvt(kT=1.0, tau=0.5,group=hoomd.group.all()) 
    hoomd.analyze.log(filename='eds.log',quantities=['momentum', 'temperature', 'time'],period=100, overwrite=True) #set up output log
    
    #equilibrate
    hoomd.run(3000)

    #simulation                  
    tfcompute.attach(nlist, r_cut=rcut,)
    hoomd.dump.gsd(filename='saveall_running.gsd',overwrite=True, period=100, group=hoomd.group.all(),dynamic=['attribute', 'momentum', 'topology'])
    hoomd.run(10000)


Note: Backed-up my_model2 previous model to my_model2/previous_model_5
notice(2): Started TF Session Manager.
notice(2): Group "all" created containing 64 particles
test
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 64
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 3000 / 3000 | TPS 7086.69 | ETA 00:00:00
Average TPS: 7074.02
---------
-- Neighborlist stats:
136 normal updates / 10 forced updates / 0 dangerous updates
n_neigh_min: 22 / n_neigh_max: 39 / n_neigh_avg: 31.7812
shortest rebuild period: 3
-- Cell list stats:
Dimension: 2, 2, 1
n_min    : 12 / n_max: 20 / n_avg: 16
** run complete **
notice(2): Force mode is FORCE_MODE.tf2hoomd 
notice(2): Starting TensorflowCompute 
notice(2): completed reallocate
notice(2): Setting flag indicating virial modification will occur
INFO:tensorflow:The following quantities will comp